In [1]:
import os
from uuid import uuid4

In [2]:

unique_id = uuid4().hex[0:8]
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"Tracing Walkthrough - {unique_id}"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
LANGCHAIN_API_KEY = os.environ.get("LANGCHAIN_API_KEY")

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

In [3]:
from langsmith import Client

client = Client()

In [4]:
from langchain import hub
from langchain.agents import AgentExecutor
from langchain.agents.format_scratchpad import format_to_openai_function_messages
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain_community.tools import DuckDuckGoSearchResults
from langchain_openai import ChatOpenAI

# Fetches the latest version of this prompt
prompt = hub.pull("wfh/langsmith-agent-prompt:5d466cbc")

llm = ChatOpenAI(
    model="gpt-3.5-turbo-16k",
    temperature=0,
)

tools = [
    DuckDuckGoSearchResults(
        name="duck_duck_go"
    ),  # General internet search using DuckDuckGo
]

llm_with_tools = llm.bind_functions(tools)

runnable_agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_function_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIFunctionsAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=runnable_agent, tools=tools, handle_parsing_errors=True
)

In [5]:
inputs = [
    "What is Tesla?",
    "What's Elon Musk's Net Worth?",
    "When was Llama-v2 released?",
    "What is the langsmith cookbook?",
    "When did langchain first announce the hub?",
]

results = agent_executor.batch([{"input": x} for x in inputs], return_exceptions=True)

In [6]:
results[:2]

[{'input': 'What is Tesla?',
  'output': "Tesla, Inc. is an American electric vehicle and clean energy company. It was founded in 2003 by a group of engineers who wanted to prove that electric cars could be better than gasoline-powered cars. Tesla is known for its innovative electric vehicles, such as the Model S, Model 3, Model X, and Model Y, as well as its energy storage products like the Powerwall and Powerpack. The company is also involved in solar energy through its acquisition of SolarCity. Tesla's mission is to accelerate the world's transition to sustainable energy."},
 {'input': "What's Elon Musk's Net Worth?",
  'output': "According to Forbes, Elon Musk's net worth is estimated to be $240.7 billion as of July 2023. Bloomberg and Forbes rank him as the world's richest person, with estimates ranging from $198 billion to $220 billion as of November 2023."}]